## Multiple Linear Regression
First we will use multiple linear regression on the data to give a baseline.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [19]:
## first load the data
coffee = pd.read_csv('../data/one_hot_coffee.csv')
coffee = coffee.copy()

In [34]:
coffee.columns
coffee.describe()

,rating,region_africa_arabia,region_caribbean,region_central_america,region_hawaii,region_asia_pacific,region_south_america,type_espresso,type_organic,type_fair_trade,type_decaffeinated,type_pod_capsule,type_blend,type_estate,Dark,Light,Medium,Medium-Dark,Medium-Light,Very Dark
count,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000,4677.000000
mean,90.459910,0.231986,0.005559,0.158435,0.018602,0.072910,0.082318,0.144323,0.089801,0.055591,0.009622,0.033782,0.085739,0.126149,0.054522,0.090229,0.291640,0.169767,0.356425,0.037417
std,3.898294,0.422145,0.074360,0.365187,0.135128,0.260016,0.274878,0.351455,0.285927,0.229155,0.097627,0.180688,0.280008,0.332053,0.227069,0.286540,0.454566,0.375468,0.478994,0.189802
min,60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,89.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,93.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,97.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [21]:
## next perform the train test split
coffee_train, coffee_test = train_test_split(coffee,
                                            shuffle=True,
                                            random_state=47,
                                            test_size = .2)

In [33]:
## make a baseline
baseline = coffee['rating'].mean()
print(baseline)

90.4599101988454


In [22]:
## import the LinearRegression object
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [23]:
predictors = ['region_africa_arabia', 'region_caribbean',
       'region_central_america', 'region_hawaii', 'region_asia_pacific',
       'region_south_america', 'type_espresso', 'type_organic',
       'type_fair_trade', 'type_decaffeinated', 'type_pod_capsule',
       'type_blend', 'type_estate', 'Dark', 'Light', 'Medium', 'Medium-Dark', 'Medium-Light', 'Very Dark']

In [24]:
reg = LinearRegression(copy_X = True)

reg.fit(coffee_train[predictors], 
        coffee_train['rating'])

LinearRegression()

In [25]:
## look at the coefficients for the fit
reg.coef_

array([ 1.88572316e+00, -1.74031760e+00,  4.36340852e-01,  9.33812529e-01,
        2.26202835e-01,  7.88241404e-02,  1.75832863e+00, -3.23183260e-02,
        9.65867934e-02,  2.67107392e-01, -1.86715742e+00,  2.22570509e-01,
        6.08682609e-01, -1.01755274e+13, -1.01755274e+13, -1.01755274e+13,
       -1.01755274e+13, -1.01755274e+13, -1.01755274e+13])

In [35]:
## make the predictions
preds = reg.predict(coffee_train[predictors])
preds_baseline = baseline * np.ones(len(coffee_train))

In [38]:
## check the mean squared error
mse = mean_squared_error(coffee_train['rating'], preds)
mse_baseline = mean_squared_error(coffee_train['rating'], preds_baseline)
print("The mean squared error for multiple linear regression is", mse)
print("The mean squared error for the baseline is", mse_baseline)

The mean squared error for simple linear regression is 9.930931500208247
The mean squared error for the baseline is 15.565458489578745


## Summary
We have made a baseline and basic model. Now we want to try lasso and ridge regression to determine which features are important and help us decide which interaction terms to include.